In [8]:
# -*- coding: utf-8
import urllib.request
import html.parser as hp

# HTMLParserを継承したクラスを定義
class Parser(hp.HTMLParser):
    result_tags = {}
    result_data = {}
    
    def make_id(self, pos):
        row_num = '%08d' % int(pos[0])
        col_num = '%08d' % int(pos[1])
        return str(row_num) + str(col_num)

    def handle_starttag(self, tag, attrs):
        """
        各タグの最初の文字列を処理する<html></html> => htmll
        """
        self.result_tags[self.make_id(self.getpos())] = [tag,attrs]

#         print("開始タグ :", tag, attrs)
#     def handle_endtag(self, tag):
#         print("終了タグ :", tag)
    def handle_data(self, data):
        """
        各タグの中身を処理する
        """
        self.result_data[self.make_id(self.getpos())] = data

        
#     def handle_comment(self, comment):
#         print("コメント:", comment)

def main():
    # 取得先URL
    url = "https://matome.naver.jp/odai/2153448009006325301"
 
    # HTMLファイルを開く
    data = urllib.request.urlopen(url)
    
    # HTMLの取得      
    html = data.read()
    html = html.decode('utf-8')

    # HTML解析(タイトルタグの値取得)     
    parser = Parser()
    parser.feed(html)
    
#     print(parser.result_tags)
    
    for k in parser.result_tags.keys(): 
        print(k)

    for v in parser.result_tags.values(): 
        print(v)

#     for tag in parser.result_tags:
#         print("result_tags")    
#         print(tag["0000208800000000"])

#     for data in parser.result_data:
#         print("result_data")    
#         print(data)

        #     print("result_data|")    
    
#     print(parser.result_data) # 

    # 終了処理
    parser.close()
    data.close()

if __name__ == "__main__":
    main()

0000000200000000
0000000300000000
0000000400000000
0000000500000000
0000000700000000
0000000900000000
0000001000000000
0000001100000000
0000001300000000
0000001500000000
0000001700000000
0000002000000000
0000002700000000
0000002800000000
0000002900000000
0000003000000000
0000003100000000
0000003200000000
0000003300000000
0000003400000000
0000003500000000
0000003600000000
0000003700000000
0000003800000000
0000003900000000
0000004000000000
0000004100000000
0000004300000000
0000004400000000
0000004500000000
0000005500000000
0000005600000000
0000005700000000
0000005900000000
0000010300000000
0000010600000000
0000011500000000
0000012200000000
0000014300000000
0000014400000000
0000014700000000
0000014800000000
0000014900000000
0000015800000000
0000015800000008
0000016000000000
0000016100000000
0000016300000000
0000016400000000
0000016500000000
0000016500000024
0000016900000000
0000017000000000
0000017100000000
0000017200000000
0000017300000000
0000017400000000
0000017600000006
00000176000000